In [ ]:
# Importación de los datos
import sqlite3
# Para importar las librerías de 'pandas' y 'numpy' es necesario instalarlas desde el terminal: "pip install pandas"
import pandas as pd
import numpy as np

## 1. CARGAR PEDIDOS (CSV)

In [ ]:
# Cargar el fichero pedidos.csv



## 2. CARGAR CLIENTES (JSON)


In [ ]:
# Cargar el fichero clientes.json



## 3. CARGAR PRODUCTOS (SQLite)


In [ ]:
# Cargar el fichero productos.db

## 4. NORMALIZAR TIPOS DE CLAVE

In [ ]:
# Convertimos los valores de las columnas que actuarán de clave a str para evitar conflictos



## 5. UNIÓN DE DATASETS

In [ ]:
# Unir los tres datasets en uno solo


## Apartado 1: Información básica del dataset


In [ ]:
# 1.1 Mostrar las primeras 5 filas del dataset global


# 1.2 Mostrar las dimensiones del dataset (filas y columnas)


# 1.3 Mostrar los nombres de todas las columnas


# 1.4 Mostrar información sobre los tipos de datos de las columnas


# 1.5 Mostrar las últimas 3 filas del dataset



## Apartado 2: Tipos de datos y valores nulos


In [ ]:
# 2.1 Mostrar información detallada del DataFrame (tipos, memoria, nulos)


# 2.2 Contar valores nulos por columna


# 2.3 Mostrar solo las columnas que tienen valores nulos


# 2.4 Calcular el porcentaje de valores nulos por columna


# 2.5 Verificar si hay filas completamente vacías



## Apartado 3: Estadísticas descriptivas


In [ ]:
# 3.1 Estadísticas descriptivas de todas las columnas numéricas


# 3.2 Estadísticas descriptivas de todas las columnas (numéricas y categóricas)


# 3.3 Calcular la media, mediana y moda de la columna 'total'


# 3.4 Calcular el rango (máximo - mínimo) de la columna 'cantidad'


# 3.5 Calcular la desviación estándar y la varianza de la columna 'total'


# 3.6 Contar valores únicos en columnas categóricas



## Apartado 4: Filtrado simple

In [ ]:
# 4.1 Filtrar pedidos con cantidad mayor a 5 unidades


# 4.2 Filtrar pedidos con total mayor a 1000 euros


# 4.3 Filtrar pedidos con precio unitario menor a 50 euros


# 4.4 Filtrar pedidos con cantidad igual a 1


# 4.5 Mostrar los primeros 5 pedidos grandes (cantidad > 5)


# 4.6 Mostrar los primeros 5 pedidos caros (total > 1000€)



## Apartado 5: Filtrado con múltiples condiciones

In [ ]:
# 5.1 Filtrar pedidos enviados con cantidad mayor a 3


# 5.2 Filtrar pedidos de tarjeta o PayPal con total mayor a 500 euros


# 5.3 Filtrar pedidos pendientes o cancelados con cantidad menor a 2


# 5.4 Filtrar pedidos enviados o devueltos con precio unitario mayor a 100


# 5.5 Mostrar los primeros 5 pedidos enviados grandes


# 5.6 Mostrar los primeros 5 pedidos premium



## Apartado 6: Filtrado temporal

In [ ]:

# 6.1 Convertir la columna fecha_pedido a tipo datetime


# 6.2 Filtrar pedidos del año 2024


# 6.3 Filtrar pedidos del mes de enero de 2024


# 6.4 Filtrar pedidos del primer trimestre de 2024


# 6.5 Filtrar pedidos de los lunes (día de la semana = 0)


# 6.6 Mostrar los primeros 5 pedidos de 2024



## Apartado 7: Estado del pedido

In [ ]:
# 7.1 Contar el número de pedidos por cada estado


# 7.2 Calcular el porcentaje de pedidos por estado


# 7.3 Calcular el total de ventas por estado


# 7.4 Calcular el valor promedio de pedidos por estado


# 7.5 Calcular la cantidad total de productos vendidos por estado


# 7.6 Mostrar estadísticas completas por estado



## Apartado 8: Método de pago

In [ ]:
# 8.1 Contar el número de pedidos por método de pago


# 8.2 Calcular el porcentaje de uso de cada método de pago


# 8.3 Calcular el valor total de ventas por método de pago


# 8.4 Calcular el valor promedio de pedidos por método de pago


# 8.5 Calcular la cantidad promedio de productos por método de pago


# 8.6 Mostrar estadísticas completas por método de pago (numero de pedidos, total de ventas, promedio de pedidos, cantidad total, promedio de la cantidad)



## Apartado 9: País de envío

In [ ]:
# 9.1 Contar el número total de países únicos


# 9.2 Mostrar los top 10 países por número de pedidos


# 9.3 Mostrar los top 10 países por valor total de ventas


# 9.4 Calcular el valor promedio de pedidos por país (top 10)


# 9.5 Calcular la cantidad total de productos enviados por país (top 10)


# 9.6 Mostrar estadísticas del país con más pedidos (número de pedidos, total de ventas, promedio de pedido y total de cantidad)





## Apartado 10: Análisis demográfico

In [ ]:
# 10.1 Contar el número total de clientes únicos


# 10.2 Distribución de clientes por género


# 10.3 Porcentaje de distribución por género


# 10.4 Distribución de clientes por nivel de fidelización


# 10.5 Estadísticas básicas de edad


# 10.6 Mostrar estadísticas completas de edad por género (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad)




## Apartado 11: Clientes por país

In [ ]:
# 11.1 Contar el número total de países únicos donde viven los clientes


# 11.2 Mostrar los top 15 países con más clientes


# 11.3 Calcular el porcentaje de clientes por país (top 10)


# 11.4 Mostrar estadísticas (número de clientes, edad promedio, edad mínima, edad máxima y mediana de edad) de edad por país (top 5 países)


# 11.5 Mostrar distribución de género por país (top 3 países)


# 11.6 Mostrar distribución de nivel de fidelización por país (top 3 países)




## Apartado 12: Clientes por ciudad

In [ ]:
# 12.1 Mostrar las top 20 ciudades con más clientes


# 12.2 Mostrar las ciudades con exactamente 1 cliente


# 12.3 Mostrar las ciudades con más de 10 clientes




## Apartado 13: Categoría de productos

In [ ]:
# 13.1 Mostrar las categorías cuyo nombre tiene más de 6 caracteres y que tienen entre 50 y 160 productos


# 13.2 Mostrar las categorías que contienen la letra "o" (mayúscula o minúscula) y tienen menos de 120 productos



## Apartado 14: Precios

In [ ]:
# 14.1 Mostrar estadísticas básicas de precios base (precio promedio, precio mínimo, precio máximo y mediana)


# 14.2 Calcular el precio promedio por categoría


# 14.3 Encontrar los productos más caros y más baratos


# 14.4 Mostrar los top 10 productos más caros




## Apartado 15: Productos más vendidos

In [ ]:

# 15.1 Obtener el producto más vendido (cantidad vendida, total vendido, nombre del producto y categoría)



## Apartado 16: Análisis temporal


In [ ]:
# 16.1 Obtener información sobre las ventas por mes (número total de ventas, número de pedidos y cantidad total)


# 16.2 Obten la misma información pero en lugar de por mes, por semana.



## Apartado 17: Duplicados

In [ ]:
## 17.1 Contar filas duplicadas en cada uno de los datasets originales.



## Apartado 18: Valores faltantes

In [ ]:
# 18.1 Obtener el porcentaje de valores faltantes en cada columna del dataset.


## Apartado 19: Valoración final

Tras completar este ejercicio, ¿qué conclusiones has obtenido acerca de los datos? ¿Consideras que sería necesario aplicar algún tipo de preprocesamiento o crees que los datos son adecuados tal como están? Apoya tu valoración con ejemplos concretos de columnas que ilustren tu análisis.

<!-- Responde aquí al apartado 19 -->
**Responder**

